# Installing Dependencies

appending torchvision reference scripts for detection to path for importing

In [ ]:
import sys
sys.path.append('../input/torchvisionreferencedetection/torchvision-reference-derection')

Installing pycocotools for evaluation

In [ ]:
!pip install pycocotools

importing required libraries

In [ ]:
import os
import torch
import torchvision
import numpy as np
import pandas as pd
import cv2
import albumentations as A
import matplotlib.pyplot as plt
from albumentations.pytorch.transforms import ToTensorV2
from engine import train_one_epoch, evaluate


# Getting Data ready

In [ ]:
train_csv_path = "../input/yolo-animal-detection-small/train.csv"
test_csv_path = "../input/yolo-animal-detection-small/test.csv"
train_images = "../input/yolo-animal-detection-small/yolo-animal-detection-small/train"
test_images = "../input/yolo-animal-detection-small/yolo-animal-detection-small/test"

In [ ]:
train_csv = pd.read_csv(train_csv_path)
train_csv.head()

In [ ]:
train_csv.shape

In [ ]:
test_csv = pd.read_csv(test_csv_path)
test_csv.head()

In [ ]:
test_csv.shape

In [ ]:
categories = train_csv["class"].unique()
print(categories)

encoding classes to integers.

- 0 is for background by default

In [ ]:
class LabelMap:
    def __init__(self, categories):
        self.map_dict = {}
        self.reverse_map_dict={}
        for i, cat in enumerate(categories):
            self.map_dict[cat] = i + 1
            self.reverse_map_dict[i] = cat
    def fit(self, df, column):
        df[column] = df[column].map(self.map_dict)
        return df
    def inverse(self, df, column):
        df[column] = df[column].map(self.map_dict)
        return df

In [ ]:
label_map = LabelMap(categories)

In [ ]:
train_csv = label_map.fit(train_csv, "class")
train_csv.head()

In [ ]:
test_csv = label_map.fit(test_csv, "class")
test_csv.head()

creating torch dataset

In [ ]:
class AnimalDataset(torch.utils.data.Dataset):
    def __init__(self, df, image_path, categories, transforms=None,**kwargs):
        super().__init__(**kwargs)
        self.df = df
        self.image_path = image_path
        self.categories = categories
        self.images = self.df["filename"].unique()
        self.transforms = transforms
    def __len__(self):
        return len(self.images)
    
    def __getitem__(self, idx):
        image_file = os.path.join(self.image_path, self.images[idx])
        img = cv2.imread(image_file)
        img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
        img = img.astype(np.float32)
        img = img/255.0
        image_data = self.df[self.df['filename'] == self.images[idx]]
        labels = torch.as_tensor(image_data["class"].values, dtype=torch.int64)
        xmins = image_data["xmin"].values
        ymins = image_data["ymin"].values
        xmaxs = image_data["xmax"].values
        ymaxs = image_data["ymax"].values
        boxes = torch.as_tensor(np.stack([xmins, ymins, xmaxs, ymaxs], axis=1), dtype=torch.float32)
        areas = (boxes[:,3] - boxes[:,1]) * (boxes[:,2] - boxes[:,0])
        areas = torch.as_tensor(areas, dtype=torch.float32)
        image_id = torch.tensor([idx])
        iscrowd = torch.zeros((len(labels),), dtype=torch.int64)
        target = {}
        target["boxes"] = boxes
        target["labels"] = labels
        target["image_id"] = image_id
        target["area"] = areas
        target["iscrowd"] = iscrowd
        if self.transforms is not None:
            transformed = self.transforms(image=img, bboxes=boxes, labels=labels)
            img = transformed["image"]
            target["boxes"] = torch.as_tensor(transformed["bboxes"],dtype=torch.float32)
        return torch.as_tensor(img, dtype=torch.float32), target
    def get_height_and_width(self, image):
        image_data = self.df.loc[self.df['filename'] == image]
        return image_data["width"].values[0], image_data["height"].values[0]

defining augmentations and transforms for training and validation

In [ ]:
transform_train = A.Compose([
    A.HorizontalFlip(p=0.5),
    A.RandomBrightnessContrast(p=0.2),
    ToTensorV2(p=1)
], bbox_params=A.BboxParams(format='pascal_voc', label_fields=['labels']))

In [ ]:
transform_test = A.Compose([
    ToTensorV2(p=1)
], bbox_params=A.BboxParams(format='pascal_voc', label_fields=['labels']))

function called after we get data from data loader

In [ ]:
def collate_fn(batch):
    return tuple(zip(*batch))

initiating datsets

In [ ]:
train_dataset = AnimalDataset(train_csv, train_images, categories, transform_train)
test_dataset = AnimalDataset(test_csv, test_images, categories, transform_test)

creating dataloaders from datasets

In [ ]:
data_loader_train = torch.utils.data.DataLoader(
        train_dataset, batch_size=4, shuffle=True, num_workers=4,
        collate_fn=collate_fn)
    
data_loader_test = torch.utils.data.DataLoader(
    test_dataset, batch_size=1, shuffle=False, num_workers=4,
    collate_fn=collate_fn)

plotting images from dataloader and verifying

In [ ]:
def plot_images(images, targets):
    for image, target in zip(images, targets):
        sample = image.permute(1,2,0).cpu().numpy()
        fig, ax = plt.subplots(1, 1, figsize=(8, 4))
        boxes = target["boxes"].cpu().numpy().astype(np.int32)
        for box in boxes:
            cv2.rectangle(sample,
                      (box[0], box[1]),
                      (box[2], box[3]),
                      (220, 0, 0), 3)
        ax.set_axis_off()
        ax.imshow(sample)

In [ ]:
images, targets = next(iter(data_loader_train))

In [ ]:
plot_images(images, targets)

In [ ]:
images, targets = next(iter(data_loader_test))

In [ ]:
plot_images(images, targets)

# Training and Validation

loading faster rcnn model from torchvision

In [ ]:
detection_model = torchvision.models.detection.fasterrcnn_resnet50_fpn(pretrained=True)

changing classification head for fine-tuning based on our dataset including background class

In [ ]:
num_classes = len(categories)+1

In [ ]:
in_features = detection_model.roi_heads.box_predictor.cls_score.in_features
detection_model.roi_heads.box_predictor = torchvision.models.detection.faster_rcnn.FastRCNNPredictor(in_features, num_classes)

load model to gpu or cpu based on device

In [ ]:
device = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")

In [ ]:
detection_model.to(device)

training and validating model in each epoch

In [ ]:
def training(model, train_loader, val_loader, epochs=10):
    # construct an optimizer
    params = [p for p in model.parameters() if p.requires_grad]
    optimizer = torch.optim.SGD(params, lr=0.005,
                                momentum=0.9, weight_decay=0.0005)
    # and a learning rate scheduler
    lr_scheduler = torch.optim.lr_scheduler.StepLR(optimizer,
                                                   step_size=3,
                                                   gamma=0.1)
    for epoch in range(epochs):
        # train for one epoch, printing every 10 iterations
        train_one_epoch(model, optimizer, train_loader, device, epoch, print_freq=10)
        # update the learning rate
        lr_scheduler.step()
        # evaluate on the test dataset
        evaluate(model, val_loader, device=device)

In [ ]:
training(detection_model, data_loader_train, data_loader_test, epochs=10 )

saving model state for further use

In [ ]:
torch.save(detection_model.state_dict(), 'fasterrcnn_resnet50_fpn.pth')